# Simple output analysis of LPJ-GUESS modeling run

In [72]:
# defaults
resolution = '1800arcsec'
regionname = 'bavaria'
model = 'chelsa-w5e5'

# receives data from snakemake
try:
    from snakemake.script import snakemake

    if snakemake.wildcards:
        print('Reading variables from snakemake variables')
        resolution = snakemake.wildcards['resolution']
        regionname = snakemake.wildcards['regionname']
        model = snakemake.wildcards['model']
    else:
        print('Note: code was run with snakemake. Executing output analysis with defaults')
except ImportError as err:
    print('Note: code was not called from within snakemake')

print('Region:', regionname, ', resolution:', resolution, ', model:', model)
outputdir = f'lpjguess_outputs_{model}_{resolution}_{regionname}/'
carbon_pools_file = outputdir + 'cpool.out'
gpp_file = outputdir + 'agpp.out'

Note: code was not called from within snakemake
Region: bavaria , resolution: 1800arcsec , model: chelsa-w5e5


In [67]:
import pandas as pd
import output_computations as comp
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "iframe"

# Total Carbon Content in vegetation

In [68]:
M2_TO_KM2 = 1_000_000
KG_TO_MT = 1_000_000

carbon_pool = pd.read_csv(carbon_pools_file, sep='\\s+').set_index(['Lon', 'Lat', 'Year'])
carbon_pool = comp.get_area_weighted_sum_per_year(carbon_pool, variable_name='VegC') / (M2_TO_KM2 * KG_TO_MT)

fig_cpool = px.line(carbon_pool.reset_index(), x="Year", y="value", title="Vegetation Carbon", labels={"value": "MtC"})
fig_cpool.show()

# Final Vegetation Carbon Content

This assumes the entire grid cells covered by vegetation.

In [61]:
print('Final carbon content (MtC): {:.2f}'.format(carbon_pool.loc[2019]))

Final carbon content (MtC): 0.41


# GPP

In [62]:
import importlib
importlib.reload(comp)
gpp = pd.read_csv(gpp_file, sep='\\s+').set_index(['Lon', 'Lat', 'Year'])
gpp = comp.get_area_weighted_mean_per_year(gpp, variable_name='Total')

fig_gpp = px.line(gpp.reset_index(), x="Year", y="value", title="Vegetation Carbon", labels={"value": 'kgC/m$^2$'})
fig_gpp.show()